In [6]:
import numpy as np
from model_functions import *
from Classification_models_LOSO_2Class import *

In [ ]:
#uncomment to run and create the all_psds file. 

#subject_data, freqs, targets = load_data(duration=30,overlap=15,seg_length=4)
#save_psds(subject_data,freqs,targets,'all_psds.npy')


In [21]:
subject_data, freqs, targets = load_psds('all_psds.npy')
test_labels = align_test_labels(classes=['A','C'])
features, targets = remove_class(subject_data,targets,'F')
features, targets = remove_test(features,targets,test_labels)
rbps = [relative_band_power(features[i],freqs,freq_bands=[0.5,4.0,7.0,9.0,11.0,13.0,25.0,45.0]) 
        for i in range(len(features))]

### k nearest neighbor classifier

In [22]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3)

In [23]:
train_metric_dict, val_metric_dict

({'acc': 0.977195196037198,
  'sens': 0.9817202669902912,
  'spec': 0.9719025651424515,
  'f1': 0.9789093816515944},
 {'acc': 0.671573438011122,
  'sens': 0.6644417475728155,
  'spec': 0.6799148332150461,
  'f1': 0.6856606136505948})

### Random Forest classifier

In [24]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, min_samples_split = 10)

In [14]:
train_metric_dict, val_metric_dict

({'acc': 0.9972078134492266,
  'sens': 0.9981687760055479,
  'spec': 0.9960838487275677,
  'f1': 0.9974122436605384},
 {'acc': 0.7474648348053647,
  'sens': 0.7809466019417476,
  'spec': 0.7083037615330021,
  'f1': 0.7692767483562463})